# Transit Boardings Report
This report includes two types of report:
- Standard: A single or comparison report for the overall scenario(s)
    - This includes sub-mode daily boardings and TOD totals (AM, MD, PM, NT) 
- Detailed: A detailed report for selected links.
    - This includes Daily and TOD boardings for user specified route(s)


In [2]:
#import required libraries
import pandas as pd
import numpy as np
import os
import glob
from functools import reduce

In [122]:
#standard or detailed?
    #detailed can include multiple routes 
routeList = [] #from file
#one or two scenarios?
bases = {'Base Model':r'G:/Regional_Modeling/1A_Archives/LRTP_2018/2040 NB Scen 01_MoDXoutputs/',
    'Comparative Model':r'G:/Regional_Modeling/1A_Archives/LRTP_2018/2016 Scen 00_08March2019_MoDXoutputs/'} #MAX 2 BASES
##note - if s2 is not None then loop before graph, if no s2 then no loop 
#   (actually maybe use loop for graph and then at the end if second map and delta, add to the layout)
##note part2- have if/else for graphing one or multiple scenarios
##note part3 - have if/else for graphing detailed or standard (also with one vs. multiple scenario)
## if detailed, aggregate by route not by meta mode.
#filter if detailed


In [103]:
#import transit assignment results
def import_transit_assignment(b):
    base = b+r'out/'
    tod = ["AM/", "MD/", "PM/", "NT/"]
    TODsums = {} # this will eventually contain all the TOD summed results

    #import csv and create sum tables for all TOD and for all day
    for x in tod:
        paths = glob.glob(os.path.join(base,x,r'*.csv'))
        tablist = []
        for p in paths:
            tablist.append(pd.read_csv(p).set_index(['ROUTE','STOP'])) #read in, set indices, and append to list
        if len(routeList) > 0: #deal with detailed filtering if needed
            for t in range(len(tablist)):
                tablist[t] = tablist[t][tablist[t].index.get_level_values('ROUTE').isin(routeList)]
        TODsums[x[0:2]] = reduce(lambda a, b: a.add(b, fill_value=0), tablist) #sum the tables for the TOD
    TODsums['daily'] = reduce(lambda a, b: a.add(b, fill_value=0), TODsums.values()) #sum of all the TOD sum tables 
    for x in TODsums.keys():
        TODsums[x] = TODsums[x].reset_index() #fix indices
    return TODsums


In [101]:
#set up route to mode to mode group table
def set_up_metamode_table(b):
    routemode = pd.read_csv(b + r'Databases/Statewide_Routes_2018S.csv', usecols=["Routes_ID", "Mode"]).drop_duplicates()

    #filter if detailed
    if len(routeList) > 0:
        routemode = routemode[routemode['Routes_ID'].isin(routeList)]
    #assign metamode to routes and modes
    routemode['metaMode'] = np.where(routemode['Mode'].isin([1,2,3]), 'MBTA_Bus',
                                    np.where(routemode['Mode'].isin([5,6,7,8]), 'Heavy_Rail',
                                            np.where(routemode['Mode'].isin([4,12,13]), 'Light_Rail',
                                                    np.where(routemode['Mode'].isin([9,32,33,34,35,36,37,38,39,40]), 'Commuter_Rail',
                                                            np.where(routemode['Mode'].isin([10,11]), 'Ferry',
                                                                    np.where(routemode['Mode'].isin([14,15,16]), 'Shuttle_Express',
                                                                            np.where(routemode['Mode'].isin([41,42,43,44,17,18,19,20,21,22]), 'RTA',
                                                                                    np.where(routemode['Mode'].isin([23,24,25,26,27,28,29,30,31]), 'Private',
                                                                                            np.where(routemode['Mode'].isin([70]), 'Walk', None)))))))))
    return routemode


In [116]:
def join_and_agg(TODSums, routemode):
#set the group by field depending on if standard or detailed report
    if len(routeList) > 0:
        agg = 'ROUTE'
    else: 
        agg = 'metaMode'

    for x in TODsums.keys():
        #join each table to route mode
        TODsums[x] = routemode.merge(TODsums[x], how='right', left_on='Routes_ID', right_on='ROUTE')
        #sum all On/Off fields by metamode 
        TODsums[x] = TODsums[x].groupby([agg])[['DirectTransferOff','DirectTransferOn','DriveAccessOn','EgressOff','Off','On',
                                                'WalkAccessOn','WalkTransferOff','WalkTransferOn']].agg('sum').reset_index()
    return TODsums

In [ ]:
#make bar charts/histograms with % of each type Direct/Drive/Walk dif colors in the bar. Put # at the top
#https://plotly.com/python/bar-charts/
#https://plotly.com/python/subplots/

In [121]:
#call things
#write a SUPER FUNCTION!!! (which calls all functions)
for g in bases.keys():
    TODsums = import_transit_assignment(bases[g])
    routemode = set_up_metamode_table(bases[g])
    TODsums = join_and_agg(TODsums, routemode)
    print(TODsums['daily'])

   ROUTE  DirectTransferOff  DirectTransferOn  DriveAccessOn   EgressOff  \
0   1601           0.038154          0.000014      24.797164   21.398636   
1   1649           5.272965         23.953019       0.000000  292.272497   
2   1654          15.725832          3.709195       0.000000   43.366414   

          Off          On  WalkAccessOn  WalkTransferOff  WalkTransferOn  
0   60.874057   60.989295      8.312079        39.437267       27.880037  
1  297.777868  297.778627    252.115631         0.232405       21.709971  
2   60.854496   60.855816     56.451973         1.762250        0.694646  
   ROUTE  DirectTransferOff  DirectTransferOn  DriveAccessOn   EgressOff  \
0   1601           0.015382          0.000466      58.011665   43.056397   
1   1649           3.735351         15.766849       0.000000  187.079020   
2   1654          11.182845          2.531834       0.000000   30.062159   

          Off          On  WalkAccessOn  WalkTransferOff  WalkTransferOn  
0   79.286299  